<a href="https://colab.research.google.com/github/YokoyamaLab/PythonBasics/blob/2024/day08_q2FromOriginalImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day08 講義内課題02: みんなの描いた数字で学習・認識

## 🌀はじめに


今日のメインパートです。機械学習を使って、画像を認識できる手順は確認しました。ただ、教科書のままでは、もともとあったデータセットで学習も認識もしているため、なんとなく達成感を持てませんよね？

という訳で、前回の宿題として皆さんが描いた0～9までの数字画像を使って学習と認識をやってみましょう。

## 🌀ステップ1: Google Driveから画像のダウンロード


皆さんの描いた画像は```/content/gdrive/Shareddrives/2024-35A01／情報科学 e／横山 昌平/img```に保存されています。このファイルを手元の環境にダウンロードしましょう。

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

# Gdrive上からGoogle Colabの環境にダウンロードする
!cp -r "/content/gdrive/Shareddrives/2024-35A01／情報科学 e／横山 昌平/img" "/content"
# 学習用データのサイズに縮小した画像を保存するディレクトリの作成
!mkdir "images"

## 🌀ステップ2: Google Driveから画像のダウンロード



教科書で扱っている画像と同等に、サイズを8ピクセル×8ピクセル、色はグレースケールで0～16の画像として保存しましょう。

In [ ]:
import os
import re
from PIL import Image
import numpy as np
import pprint

# リサイズ済み画像(学習に食わせる画像)の置き場
dir = "/content/images/"
# 大きな元画像の置き場
dir_original = "/content/img/"

# ファイル名の一覧を取得（この講義では教えていないPython特有の書き方をしてみました）
files = [
    f for f in os.listdir(dir_original) if os.path.isfile(os.path.join(dir_original, f))
]
# とりあえずファイル一覧を表示してみる
print(files)

# 学習用画像のサイズ(一辺のピクセル数)
size = 8

# 数字毎にファイルパスを格納しておく辞書型の変数を準備する
images = []
numbers =  []
sources = []
# originalディレクトリの画像ファイルを縮小してimagesに入れる
for file in files:
  try:
    # ファイルを開く
    img = Image.open(dir_original + file)
    # ファイルを指定されたサイズでリサイズし白黒化する
    img_resized = img.resize((size, size),Image.ANTIALIAS).convert('L')
    # 小さくしたファイルを保存する
    img_resized.save(dir + file, quality=90)
    # 正規表現で学籍番号、番号、拡張子に分割する
    student,number,extension = re.split('[_.]', file)
    # 画像は二次元配列なのでこれを一次元化しimagesに追加
    images.append(np.asarray(img_resized).flatten())
    # 画像が表す数もnumbersに追加
    numbers = np.append(numbers,int(number))
    # エラー無いものだけsoucesにファイルパス追加
    sources.append(file)
  # PILで画像として読めないファイルだった時
  except Image.UnidentifiedImageError:
    print(file,"読めない画像形式なのでスキップします")

# imagesをnumPy配列に変換
images =  np.asarray(images,dtype=float)
# このままだと0～255のグレースケール画像なので、
# 教科書で使っているsklearn.datasetsのload_digitsと同様に0(黒)～16(白)に変換
images = 16 - np.floor(17*images/256)


## 🌀ステップ3:画像を確認してみる。


どんな画像があるのか、さっと一覧で出してみましょう。

In [ ]:
import matplotlib.pyplot as plt

col = 10
fig, axes = plt.subplots( 1+int(np.ceil(len(images)/col)),col, figsize=(10,10))
#plt.subplots_adjust(wspace=2, hspace=2)
for r in range(int(np.ceil(len(images)/col))):
  row = axes[r]
  for c in range(col):
    if r*col+c >= len(images):
      break;
    # ファイルを開く
    img = Image.open(dir + sources[r*col+c])
    row[c].imshow(img,cmap="gray")
    row[c].axis('off')
    #row[c].set_title(int(numbers[r*col+c]))
plt.show()



## 🌀ステップ4:データを訓練データとテストデータに分ける


機械学習を試行する場合、手持ちのデータを訓練データとテストデータに分けるのが一般です。訓練データのみを使って、機械学習し、なんらかの判定器を作り、訓練に使ったデータとは別のデータを判定させる事で、未知りデータに対してどのような精度が出るのかを明らかにします。

In [ ]:
from sklearn.model_selection import train_test_split

# ここで『Xからyを予測する』という機械学習に取り組む
# つまり画像(を配列にしたもの)から画像に書かれている数字を予測する
X = images
y = numbers

# 訓練データとテストデータに分ける(学習した画像とは違う画像で制度を検証するため)
X_train,X_test,y_train,y_test= train_test_split(X,y)
# 何も指定しないと25%がテストデータとなり75%が訓練データとなる
# 以下のように明示的に割合を変える事も可能
# X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.6)

print("訓練データ数: ", len(X_train))
print("テストデータ数: ",len(X_test))

np.savetxt('out.csv',X_train,delimiter=',')

## 🌀ステップ5:SVMを使い数字画像の判定器を作り画像を判定する


では実際に機械学習で【数字画像の判定器】を作成し、それをテストデータに適用し正解率を出してみましょう。（テストデータのつくまり方によって毎回正解率が多少変化します）

In [ ]:
# 教科書p.180のコードを使って、自前の画像データで
# 学習・推定し、正解率を計測しよう
from sklearn import svm
from sklearn.metrics import accuracy_score

model = svm.SVC(kernel="rbf", gamma=0.001)
model.fit(X_train,y_train)

pred = model.predict(X_test)
score = accuracy_score(y_test, pred)

print("正解率: ",score*100,"%")

## 🌀ステップ6:sklearnのテストデータも追加しましょう

訓練データはもっと沢山あっても良いので、教科書にあったsklearnのデータセットも追加してみましょう。

In [ ]:
import pandas as pd
from sklearn.datasets import load_digits

sklearn_dataset = load_digits()
sklearn_images = sklearn_dataset.data
sklearn_numbers = sklearn_dataset.target

X_train_plus = np.concatenate([X_train, sklearn_images])
y_train_plus = np.concatenate([y_train, sklearn_numbers])


これでもう一度試します。

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

model = svm.SVC(kernel="rbf", gamma=0.001)
model.fit(X_train_plus,y_train_plus)

pred = model.predict(X_test)
score = accuracy_score(y_test, pred)

print("正解率: ",score*100,"%")


正解率は上がりましたか？下がりましたか？（必ずしもデータ数がおおいからといって正解率が上がるわけではない事に注意してください。）

さて、次に認識が違った画像のみを出してみましょう。どうでしょうか。認識が間違うだけの理由がありそうですか？

In [ ]:
dame = 0
for xt in range(len(X_test)):
  pred = model.predict([X_test[xt]])
  if pred[0] != y_test[xt]:
    print("【間違い！】推定:",int(pred[0]),"正解:",int(y_test[xt]))
    img = X_test[xt].reshape([8, 8])
    plt.imshow(img,cmap="gray")
    plt.show()
    dame += 1
print(len(X_test),"画像中",dame,"個ダメ！","正解率:",(len(X_test)-dame)/len(X_test),"%")

## 🌀課題！　未知の画像の数字予測

このノートでトレーニングした判定器を使って、全く別の画像を判定してみましょう。googleドライブのimg2ディレクトリに、手書きではなくコンピュータで描いた数字の画像が3セットあります。

まずは、これをローカルにもってきます。

In [ ]:
# Gdrive上からGoogle Colabの環境にダウンロードする
!cp -r "/content/gdrive/Shareddrives/2024-35A01／情報科学 e／横山 昌平/img2" "/content"
# 学習用データのサイズに縮小した画像を保存するディレクトリの作成
!mkdir "images2"

さて、これを判定してみましょう。これを課題とします。

In [ ]:
## コードをここに